<a href="https://colab.research.google.com/github/Amr0093/Amr0093/blob/main/4_Evaluating_language_knowledge_of_ELL_students_from_grades_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Bidirectional, Flatten, Dense
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import plotly.express as exp
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import os
user = 'amrahmed93'
key = 'e046f1a9930f5caf3b73e3d4c96317b2'
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c feedback-prize-english-language-learning -p /content/sample_data/English_Learning_Prize

feedback-prize-english-language-learning.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/sample_data/English_Learning_Prize/feedback-prize-english-language-learning.zip -d /content/sample_data/English_Learning_Prize

Archive:  /content/sample_data/English_Learning_Prize/feedback-prize-english-language-learning.zip
replace /content/sample_data/English_Learning_Prize/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
train_df = pd.read_csv('/content/sample_data/English_Learning_Prize/train.csv')
test_df  =pd.read_csv('/content/sample_data/English_Learning_Prize/test.csv')

In [ ]:
type(train_df)

pandas.core.frame.DataFrame

In [ ]:
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [ ]:
test_df.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


Split the data set to X,y

In [ ]:
X = train_df['full_text'].values
y = train_df[['cohesion', 'syntax', 'vocabulary', 'phraseology','grammar', 'conventions']].values

In [ ]:
length= len(train_df['full_text'].values)
Max = train_df['full_text'].values[0]
for i,element in enumerate(train_df['full_text'].values):
  if len(element) > len(Max):
    Max = element
print(f'the input list has a length of : {length} and a width of : {len(Max)}')

the input list has a length of : 3911 and a width of : 6044


https://www.geeksforgeeks.org/nlp-how-tokenizing-text-sentence-words-works/

https://stackoverflow.com/questions/61656444/warning-warningtensorflowmodel-was-constructed-with-shape-none-150-but-i

In [ ]:
def text_preprocess(data_response):
    #lower case
    response_lower=[text.lower() for text in data_response]
    #remove hyperlink
    response_link_rem = [re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', text) for text in response_lower]
    #remove url
    response_url_rem = [re.sub(r"(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'"".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))", '', text) for text in response_link_rem]
    #remove special character
    response_sp_rem = [re.sub(r"[^A-Za-z]+", ' ', text) for text in response_url_rem]
    #Tokenize into words
    response_token = [word_tokenize(text) for text in response_sp_rem]
    #Remove Stop words
    stopword = list(set(stopwords.words('english'))-set(['not']))
    response_stop_rem=[]
    for text in response_token:
        response_stop_rem.append([word for word in text if word not in stopword])
    #Lemmatize the word
    response_lem=[]
    for text in response_stop_rem:
        le=WordNetLemmatizer()
        response_lem.append([le.lemmatize(word) for word in text])
    #Final Cleaned response
    cleaned_response=[' '.join(text) for text in response_lem]
    #convert to sequences
    tokenizer = Tokenizer(num_words=20000)
    sequence = tokenizer.texts_to_sequences(cleaned_response)
    train_x = pad_sequences(sequence, maxlen=200)
    return train_x


In [ ]:
# Defining Train and Validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.3)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
X_train = pd.DataFrame(X_train, columns =['full_text'])
y_train = pd.DataFrame(y_train, columns =['cohesion',	'syntax',	'vocabulary',	'phraseology',	'grammar',	'conventions'])
X_val = pd.DataFrame(X_val, columns =['full_text'])
y_val = pd.DataFrame(y_val, columns =['cohesion',	'syntax',	'vocabulary',	'phraseology',	'grammar',	'conventions'])

In [ ]:
X_train = text_preprocess(X_train['full_text'].values)
Y_train=y_train[['cohesion',	'syntax',	'vocabulary',	'phraseology',	'grammar',	'conventions']].iloc[:,0]
X_val = text_preprocess(X_val['full_text'].values)
Y_val=y_val[['cohesion',	'syntax',	'vocabulary',	'phraseology',	'grammar',	'conventions']].iloc[:,0]
X_train.shape, Y_train.shape, X_val.shape,Y_val.shape

((2737, 200), (2737,), (1174, 200), (1174,))

In [ ]:
# type(Y_train)
type(Y_train.iloc[:,0])  # pandas Series
Y_train.iloc[:,0].shape

(2737,)

In [ ]:
#Model Building
model=Sequential()
model.add(Embedding(20000, 64, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(64, activation='softmax')))

model.add(Flatten())

model.add(Dense(512, activation='softmax'))

model.add(Dense(512, activation='softmax'))

model.add(Dense(512, activation='softmax'))


model.add(Dense(1, activation='softmax'))

In [ ]:
#Model Summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 200, 64)           1280000   
                                                                 
 bidirectional_7 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 512)               66048     
                                                                 
 dense_29 (Dense)            (None, 512)               262656    
                                                                 
 dense_30 (Dense)            (None, 512)               262656    
                                                      

In [ ]:
#Model compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [ ]:
#you could have used the following code instead of all the hassle of splitting the data set
# history = model.fit([x, y], output, epochs=20, batch_size=32, validation_split=3/9)

In [ ]:
#Model Fitting and Evaluation
history = model.fit(X_train, Y_train,epochs=5, batch_size=10, validation_data=(X_val, Y_val))

Epoch 1/5
274/274 [==============================] - 55s 200ms/step - loss: -0.1827 - accuracy: 0.0026 - val_loss: -0.9186 - val_accuracy: 0.0026
Epoch 2/5
274/274 [==============================] - 54s 198ms/step - loss: -1.6465 - accuracy: 0.0026 - val_loss: -2.4118 - val_accuracy: 0.0026
Epoch 3/5
274/274 [==============================] - 55s 199ms/step - loss: -3.1553 - accuracy: 0.0026 - val_loss: -3.9433 - val_accuracy: 0.0026
Epoch 4/5
274/274 [==============================] - 53s 194ms/step - loss: -4.6771 - accuracy: 0.0026 - val_loss: -5.4737 - val_accuracy: 0.0026
Epoch 5/5
274/274 [==============================] - 54s 198ms/step - loss: -6.2267 - accuracy: 0.0026 - val_loss: -7.0764 - val_accuracy: 0.0026


In [ ]:
exp.line(data_frame=history.history, y=['accuracy', 'val_accuracy'])

## Building Machine Learning Model and Evaluation

In [ ]:
mnb = MultinomialNB().fit(X_train, Y_train)

In [ ]:
pred = mnb.predict(X_val)

In [ ]:
accuracy_score(pred, Y_val)